## Libraries

In [1]:
import pandas as pd
import numpy as np
import math
import requests
import os
import zipfile
import seaborn as sns
import matplotlib.pyplot as plt
from random import random
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder


## Download TSE files

In [2]:
# set current directory and change to folder "raw_data"
current_dir_orig = os.getcwd()
os.chdir(f"{current_dir_orig}/raw_data")

In [3]:
def download_files(zipp):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
    
    for file_name, url in zipp:
        
        download = requests.get(url, headers=headers, stream=True)
        open(f"{file_name}", 'wb').write(download.content)
    

url_prest_contas = 'https://cdn.tse.jus.br/estatistica/sead/odsele/prestacao_contas/prestacao_de_contas_eleitorais_candidatos_2022.zip'
url_candidatos = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_cand/consulta_cand_2022.zip'
url_vagas = 'https://cdn.tse.jus.br/estatistica/sead/odsele/consulta_vagas/consulta_vagas_2022.zip'

zipfiles_names = ['prestacao_de_contas_eleitorais_candidatos_2022.zip', 'consulta_cand_2022.zip', 'consulta_vagas_2022.zip']
urls = [url_prest_contas, url_candidatos, url_vagas]

current_dir = os.getcwd()

# criteria to download only once
if len(os.listdir(current_dir)) == 0:
    download_files(zip(zipfiles_names, urls))
else:
    pass


## Import Data

In [4]:
files = os.listdir(current_dir)

zip_list = [zipfile.ZipFile(file) for file in files]

candidatos_relevant_files = ['consulta_cand_2022_BRASIL.csv']
vagas_relevant_files = ['consulta_vagas_2022_BRASIL.csv']
prestcontas_relevant_files = ['receitas_candidatos_2022_BRASIL.csv', 'despesas_contratadas_candidatos_2022_BRASIL.csv']


In [5]:
# columns of interest

cols_candidatos = ['ANO_ELEICAO',
                   'DS_ELEICAO',
                   'SG_UE',
                   'DS_CARGO',
                   'SQ_CANDIDATO',
                   'NR_CANDIDATO',
                   'NM_CANDIDATO',
                   'NR_PARTIDO',
                   'SG_PARTIDO',
                   'NM_PARTIDO',
                   'DS_NACIONALIDADE',
                   'NR_IDADE_DATA_POSSE',
                   'DS_GENERO',
                   'DS_GRAU_INSTRUCAO',
                   'DS_ESTADO_CIVIL',
                   'DS_COR_RACA',
                   'DS_OCUPACAO',
                   'DS_SIT_TOT_TURNO',
                   'ST_CANDIDATO_INSERIDO_URNA',
                   'ST_REELEICAO'
                   ]

cols_vagas = ['ANO_ELEICAO',
              'DS_ELEICAO',
              'SG_UE',
              'DS_CARGO',
              'QT_VAGAS'
             ]


cols_receitas = ['ANO_ELEICAO',
               'DS_ELEICAO',
               'TP_PRESTACAO_CONTAS',
               'SQ_PRESTADOR_CONTAS',
               'SG_UE',
               'NR_CNPJ_PRESTADOR_CONTA',
               'DS_CARGO',
               'SQ_CANDIDATO',
               'NR_CANDIDATO',
               'NM_CANDIDATO',
               'NR_PARTIDO',
               'SG_PARTIDO',
               'NM_PARTIDO',
               'DS_FONTE_RECEITA',
               'DS_ORIGEM_RECEITA',
               'DS_ESPECIE_RECEITA',
               'NR_CPF_CNPJ_DOADOR',
               'NM_DOADOR_RFB',
               'DT_RECEITA',
               'VR_RECEITA'
               ]

cols_despesas = ['ANO_ELEICAO',
                'DS_ELEICAO',
                'TP_PRESTACAO_CONTAS',
                'SQ_PRESTADOR_CONTAS',
                'SG_UE',
                'NR_CNPJ_PRESTADOR_CONTA',
                'DS_CARGO',
                'SQ_CANDIDATO',
                'NR_CANDIDATO',
                'NM_CANDIDATO',
                'NR_PARTIDO',
                'SG_PARTIDO',
                'NM_PARTIDO',
                'DS_TIPO_FORNECEDOR',
                'NR_CPF_CNPJ_FORNECEDOR',
                'NM_FORNECEDOR_RFB',
                'DS_ORIGEM_DESPESA',
                'DT_DESPESA',
                'VR_DESPESA_CONTRATADA'
               ]

cols_eleitos = ['ds_cargo',
                 'nr_candidato',
                 'nm_candidato',
                 'sg_partido',
                 'sg_ue',
                 'sq_candidato',
                 'qt_votos_nom_validos'
                ]


candidatos_df = pd.read_csv(zip_list[0].open(candidatos_relevant_files[0]),
                            sep = ';',
                            usecols = cols_candidatos,
                            encoding = 'windows-1252'
                            )

vagas_df = pd.read_csv(zip_list[1].open(vagas_relevant_files[0]),
                       sep = ';',
                       usecols = cols_vagas,
                       encoding = 'windows-1252'
                       )

receitas_df = pd.read_csv(zip_list[2].open(prestcontas_relevant_files[0]),
                        sep = ';',
                        usecols = cols_receitas,
                        encoding = 'windows-1252'
                        )

despesas_df = pd.read_csv(zip_list[2].open(prestcontas_relevant_files[1]),
                         sep = ';',
                         usecols = cols_despesas,
                         encoding = 'windows-1252'
                         )

# change to original directory
os.chdir(current_dir_orig)

eleitos_df = pd.read_csv(r'./support_files/eleitas_eleitos-região_presidente_2022.csv',
                         sep = ';',
                         usecols = cols_eleitos,
                         encoding = 'windows-1252'
                         )

depfed_redes_sociais_df = pd.read_excel(r'./support_files/depfed_numeros_redesocial.xlsx')

depfed_quociente_eleitoral = pd.read_csv(r'./support_files/quociente_eleitoral-brasil_deputado federal_2022.csv',
                                         sep = ';',
                                         encoding = 'windows-1252'
                                         )


## Exploratory Data Analysis

In [6]:
# see all columns
pd.set_option('display.max_columns', None)

In [7]:
candidatos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29314 entries, 0 to 29313
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ANO_ELEICAO                 29314 non-null  int64  
 1   DS_ELEICAO                  29314 non-null  object 
 2   SG_UE                       29314 non-null  object 
 3   DS_CARGO                    29314 non-null  object 
 4   SQ_CANDIDATO                29314 non-null  int64  
 5   NR_CANDIDATO                29314 non-null  int64  
 6   NM_CANDIDATO                29314 non-null  object 
 7   NR_PARTIDO                  29314 non-null  int64  
 8   SG_PARTIDO                  29314 non-null  object 
 9   NM_PARTIDO                  29314 non-null  object 
 10  DS_NACIONALIDADE            29314 non-null  object 
 11  NR_IDADE_DATA_POSSE         29289 non-null  float64
 12  DS_GENERO                   29314 non-null  object 
 13  DS_GRAU_INSTRUCAO           293

In [8]:
candidatos_df.head()

,ANO_ELEICAO,DS_ELEICAO,SG_UE,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DS_NACIONALIDADE,NR_IDADE_DATA_POSSE,DS_GENERO,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_CANDIDATO_INSERIDO_URNA
0,2022,Eleições Gerais Estaduais 2022,DF,DEPUTADO FEDERAL,70001650385,1123,CELY LANGAMER MUNIZ,11,PP,PROGRESSISTAS,BRASILEIRA NATA,51.0,FEMININO,SUPERIOR COMPLETO,CASADO(A),BRANCA,PROFESSOR DE ENSINO DE PRIMEIRO E SEGUNDO GRAUS,NÃO ELEITO,N,SIM
1,2022,Eleições Gerais Estaduais 2022,PE,DEPUTADO FEDERAL,170001618619,1167,MARCOS BEZERRA CAMPELO,11,PP,PROGRESSISTAS,BRASILEIRA NATA,55.0,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,OUTROS,SUPLENTE,N,SIM
2,2022,Eleições Gerais Estaduais 2022,RO,DEPUTADO ESTADUAL,220001647615,35355,DIHESSICA DAYANE FERREIRA DA SILVA,35,PMB,PARTIDO DA MULHER BRASILEIRA,BRASILEIRA NATA,25.0,FEMININO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),PARDA,OUTROS,NÃO ELEITO,N,SIM
3,2022,Eleições Gerais Estaduais 2022,RO,DEPUTADO FEDERAL,220001605554,4456,MARY TERESINHA BRAGANHOL,44,UNIÃO,UNIÃO BRASIL,BRASILEIRA NATA,58.0,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),BRANCA,SERVIDOR PÚBLICO ESTADUAL,SUPLENTE,N,SIM
4,2022,Eleições Gerais Estaduais 2022,RJ,DEPUTADO ESTADUAL,190001613885,33337,ROBERTA PATRÍCIA DA SILVA ALMEIDA,33,PMN,PARTIDO DA MOBILIZAÇÃO NACIONAL,BRASILEIRA NATA,47.0,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),BRANCA,DONA DE CASA,SUPLENTE,N,SIM


In [9]:
vagas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ANO_ELEICAO  109 non-null    int64 
 1   DS_ELEICAO   109 non-null    object
 2   SG_UE        109 non-null    object
 3   DS_CARGO     109 non-null    object
 4   QT_VAGAS     109 non-null    int64 
dtypes: int64(2), object(3)
memory usage: 4.4+ KB


In [10]:
vagas_df.head()

,ANO_ELEICAO,DS_ELEICAO,SG_UE,DS_CARGO,QT_VAGAS
0,2022,Eleição Geral Federal 2022,BR,Presidente,1
1,2022,Eleições Gerais Estaduais 2022,AL,Governador,1
2,2022,Eleições Gerais Estaduais 2022,AL,Senador,1
3,2022,Eleições Gerais Estaduais 2022,AL,Deputado Federal,9
4,2022,Eleições Gerais Estaduais 2022,AL,Deputado Estadual,27


In [11]:
receitas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653273 entries, 0 to 653272
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   ANO_ELEICAO              653273 non-null  int64 
 1   DS_ELEICAO               653273 non-null  object
 2   TP_PRESTACAO_CONTAS      653273 non-null  object
 3   SQ_PRESTADOR_CONTAS      653273 non-null  int64 
 4   SG_UE                    653273 non-null  object
 5   NR_CNPJ_PRESTADOR_CONTA  653273 non-null  int64 
 6   DS_CARGO                 653273 non-null  object
 7   SQ_CANDIDATO             653273 non-null  int64 
 8   NR_CANDIDATO             653273 non-null  int64 
 9   NM_CANDIDATO             653273 non-null  object
 10  NR_PARTIDO               653273 non-null  int64 
 11  SG_PARTIDO               653273 non-null  object
 12  NM_PARTIDO               653273 non-null  object
 13  DS_FONTE_RECEITA         653273 non-null  object
 14  DS_ORIGEM_RECEITA   

In [12]:
print(receitas_df.shape)
receitas_df.head()

(653273, 20)


,ANO_ELEICAO,DS_ELEICAO,TP_PRESTACAO_CONTAS,SQ_PRESTADOR_CONTAS,SG_UE,NR_CNPJ_PRESTADOR_CONTA,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DS_FONTE_RECEITA,DS_ORIGEM_RECEITA,DS_ESPECIE_RECEITA,NR_CPF_CNPJ_DOADOR,NM_DOADOR_RFB,DT_RECEITA,VR_RECEITA
0,2022,Eleições Gerais Estaduais 2022,RELATÓRIO FINANCEIRO,3796037695,SP,47552212000145,Deputado Federal,250001651724,4427,REINALDO MONTEIRO DA SILVA,44,UNIÃO,UNIÃO BRASIL,OUTROS RECURSOS,Recursos próprios,PIX,16653258874,REINALDO MONTEIRO DA SILVA,29/08/2022,"461,75"
1,2022,Eleições Gerais Estaduais 2022,RELATÓRIO FINANCEIRO,3796037695,SP,47552212000145,Deputado Federal,250001651724,4427,REINALDO MONTEIRO DA SILVA,44,UNIÃO,UNIÃO BRASIL,OUTROS RECURSOS,Recursos próprios,PIX,16653258874,REINALDO MONTEIRO DA SILVA,22/08/2022,"187,30"
2,2022,Eleições Gerais Estaduais 2022,RELATÓRIO FINANCEIRO,3785019283,SC,47497189000133,Deputado Estadual,240001616209,44117,MARIA CRISTINA CORREA CLEMENTE,44,UNIÃO,UNIÃO BRASIL,OUTROS RECURSOS,Recursos próprios,PIX,90908910991,MARIA CRISTINA CORREA CLEMENTE,18/08/2022,"400,00"
3,2022,Eleições Gerais Estaduais 2022,RELATÓRIO FINANCEIRO,3785019283,SC,47497189000133,Deputado Estadual,240001616209,44117,MARIA CRISTINA CORREA CLEMENTE,44,UNIÃO,UNIÃO BRASIL,OUTROS RECURSOS,Recursos próprios,PIX,90908910991,MARIA CRISTINA CORREA CLEMENTE,18/08/2022,"200,00"
4,2022,Eleições Gerais Estaduais 2022,RELATÓRIO FINANCEIRO,3778701223,MS,47464846000146,Deputado Federal,120001609244,4444,SINDOLEY LUIZ DE SOUZA MORAIS,44,UNIÃO,UNIÃO BRASIL,OUTROS RECURSOS,Recursos próprios,Transferência eletrônica,97652911168,SINDOLEY LUIZ DE SOUZA MORAIS,09/09/2022,"2511,00"


In [13]:
despesas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994977 entries, 0 to 1994976
Data columns (total 19 columns):
 #   Column                   Dtype 
---  ------                   ----- 
 0   ANO_ELEICAO              int64 
 1   DS_ELEICAO               object
 2   TP_PRESTACAO_CONTAS      object
 3   SQ_PRESTADOR_CONTAS      int64 
 4   SG_UE                    object
 5   NR_CNPJ_PRESTADOR_CONTA  int64 
 6   DS_CARGO                 object
 7   SQ_CANDIDATO             int64 
 8   NR_CANDIDATO             int64 
 9   NM_CANDIDATO             object
 10  NR_PARTIDO               int64 
 11  SG_PARTIDO               object
 12  NM_PARTIDO               object
 13  DS_TIPO_FORNECEDOR       object
 14  NR_CPF_CNPJ_FORNECEDOR   int64 
 15  NM_FORNECEDOR_RFB        object
 16  DS_ORIGEM_DESPESA        object
 17  DT_DESPESA               object
 18  VR_DESPESA_CONTRATADA    object
dtypes: int64(7), object(12)
memory usage: 289.2+ MB


In [14]:
print(despesas_df.shape)
despesas_df.head()

(1994977, 19)


,ANO_ELEICAO,DS_ELEICAO,TP_PRESTACAO_CONTAS,SQ_PRESTADOR_CONTAS,SG_UE,NR_CNPJ_PRESTADOR_CONTA,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DS_TIPO_FORNECEDOR,NR_CPF_CNPJ_FORNECEDOR,NM_FORNECEDOR_RFB,DS_ORIGEM_DESPESA,DT_DESPESA,VR_DESPESA_CONTRATADA
0,2022,Eleições Gerais Estaduais 2022,Final,3793673304,AC,47547980000100,Deputado Federal,10001642333,1515,JESSICA ROJAS SALES,15,MDB,Movimento Democrático Brasileiro,Pessoa Física,65149203220,AMARILIO SARAIVA DE OLIVEIRA,Atividades de militância e mobilização de rua,22/08/2022,"4000,00"
1,2022,Eleições Gerais Estaduais 2022,Final,3783868250,AC,47493579000135,Deputado Federal,10001613550,1331,LEONARDO CUNHA DE BRITO,13,PT,Partido dos Trabalhadores,Pessoa Física,6457176224,IRANILDO MATIAS DOS SANTOS,Atividades de militância e mobilização de rua,03/09/2022,"500,00"
2,2022,Eleições Gerais Estaduais 2022,Final,3783868250,AC,47493579000135,Deputado Federal,10001613550,1331,LEONARDO CUNHA DE BRITO,13,PT,Partido dos Trabalhadores,Pessoa Física,6457176224,IRANILDO MATIAS DOS SANTOS,Atividades de militância e mobilização de rua,03/09/2022,"500,00"
3,2022,Eleições Gerais Estaduais 2022,Final,3783868250,AC,47493579000135,Deputado Federal,10001613550,1331,LEONARDO CUNHA DE BRITO,13,PT,Partido dos Trabalhadores,Pessoa Física,6457176224,IRANILDO MATIAS DOS SANTOS,Atividades de militância e mobilização de rua,03/09/2022,"500,00"
4,2022,Eleições Gerais Estaduais 2022,Final,3791359992,AC,47529158000117,Deputado Estadual,10001635541,10000,JAMYL ASFURY MARTINS OLIVEIRA,10,REPUBLICANOS,REPUBLICANOS,Pessoa Física,1822169267,UEBERTON JUNIOR NAVA DA SILVA,Atividades de militância e mobilização de rua,18/08/2022,"1200,00"


In [15]:
eleitos_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513 entries, 0 to 512
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ds_cargo              513 non-null    object
 1   nr_candidato          513 non-null    int64 
 2   nm_candidato          513 non-null    object
 3   sg_partido            513 non-null    object
 4   sg_ue                 513 non-null    object
 5   sq_candidato          513 non-null    int64 
 6   qt_votos_nom_validos  513 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 28.2+ KB


In [16]:
print(eleitos_df.shape)
eleitos_df.head()

(513, 7)


,ds_cargo,nr_candidato,nm_candidato,sg_partido,sg_ue,sq_candidato,qt_votos_nom_validos
0,Deputado Federal,1101,MARIA DO SOCORRO NERI MEDEIROS DE SOUZA,PP,AC,10001621643,25842
1,Deputado Federal,4455,ROZIMEIRE RIBEIRO ANDRADE,UNIÃO,AC,10001612486,21285
2,Deputado Federal,4422,ULYSSES FREITAS PEREIRA DE ARAUJO,UNIÃO,AC,10001612487,21075
3,Deputado Federal,1122,JOSÉ ESTEPHAN BARBARY FILHO,PP,AC,10001621646,19958
4,Deputado Federal,1111,GEHLEN DINIZ ANDRADE,PP,AC,10001621650,19560


In [17]:
depfed_redes_sociais_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9970 entries, 0 to 9969
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   SQ_CANDIDATO          9970 non-null   float64
 1   NM_CANDIDATO          9970 non-null   object 
 2   NM_URNA_CANDIDATO     9969 non-null   object 
 3   DS_SIT_TOT_TURNO      9970 non-null   object 
 4   SG_PARTIDO            9708 non-null   object 
 5   SG_UE                 9708 non-null   object 
 6   QT_VOTOS_NOM_VALIDOS  9708 non-null   float64
 7   URL_INSTAGRAM         5286 non-null   object 
 8   INSTA_FLW             9708 non-null   float64
 9   URL_TIKTOK            1561 non-null   object 
 10  TK_FLW                9949 non-null   float64
 11  TK_LIKE               2897 non-null   float64
 12  URL_FACEBOOK          4975 non-null   object 
 13  FB_FLW                9707 non-null   float64
 14  URL_TWITTER           2126 non-null   object 
 15  TWT_FLW              

In [18]:
print(depfed_redes_sociais_df.shape)
depfed_redes_sociais_df.head()

(9970, 16)


,SQ_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,DS_SIT_TOT_TURNO,SG_PARTIDO,SG_UE,QT_VOTOS_NOM_VALIDOS,URL_INSTAGRAM,INSTA_FLW,URL_TIKTOK,TK_FLW,TK_LIKE,URL_FACEBOOK,FB_FLW,URL_TWITTER,TWT_FLW
0,7.000162e+10,BEATRIZ KICIS TORRENTS DE SORDI,BIA KICIS,ELEITO POR QP,PL,DF,214733.0,https://www.instagram.com/biakicis/,1500000.0,NaN,0.0,0.0,https://www.facebook.com/biakicisoficial/,1500000.0,https://twitter.com/Biakicis,1700000.0
1,1.700016e+11,ANDRÉ FERREIRA RODRIGUES,ANDRÉ FERREIRA,ELEITO POR QP,PL,PE,273267.0,https://www.instagram.com/andreferreira.pe/,48400.0,https://www.tiktok.com/@andreferreira.pe,847.0,6355.0,https://www.facebook.com/andreferreira.pe/,54000.0,https://mobile.twitter.com/aferreira2222,286.0
2,2.200016e+11,FERNANDO RODRIGUES MÁXIMO,DR FERNANDO MÁXIMO,ELEITO POR QP,UNIÃO,RO,85604.0,https://instagram.com/drfernandomaximo007,37800.0,https://www.tiktok.com/@drfernandomaximo,276.0,511.0,https://facebook.com/drfernandomaximo007,30000.0,NaN,0.0
3,1.900016e+11,DANIELA MOTÉ DE SOUZA CARNEIRO,DANIELA DO WAGUINHO,ELEITO POR QP,UNIÃO,RJ,213706.0,https://www.instagram.com/deputadadanieladowag...,47000.0,NaN,0.0,0.0,https://pt-br.facebook.com/danieladowaguinho,35000.0,https://twitter.com/danielawaguinho,3837.0
4,1.300016e+11,NIKOLAS FERREIRA DE OLIVEIRA,NIKOLAS FERREIRA,ELEITO POR QP,PL,MG,1492047.0,https://www.instagram.com/nikolasferreiradm/,6000000.0,https://www.tiktok.com/@nikolasferreiradm,3500000.0,48500000.0,https://www.facebook.com/nikolasferreiradm/,4000000.0,https://twitter.com/nikolas_dm,1000000.0


In [19]:
depfed_quociente_eleitoral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1355 entries, 0 to 1354
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   aa_eleicao              1355 non-null   int64 
 1   sg_uf                   1355 non-null   object
 2   cd_cargo                1355 non-null   int64 
 3   nm_cargo                1355 non-null   object
 4   qt_votos_validos_qe     1355 non-null   int64 
 5   qt_vagas_qe             1355 non-null   int64 
 6   vr_quociente_eleitoral  1355 non-null   int64 
 7   tx_legenda_tot          1355 non-null   object
dtypes: int64(5), object(3)
memory usage: 84.8+ KB


In [20]:
print(depfed_quociente_eleitoral.shape)
depfed_quociente_eleitoral.head()

(1355, 8)


,aa_eleicao,sg_uf,cd_cargo,nm_cargo,qt_votos_validos_qe,qt_vagas_qe,vr_quociente_eleitoral,tx_legenda_tot
0,2014,AC,6,Deputado Federal,399201,8,49900,PMDB / PP / PPS / PR / PSC / PSD / PSDB / PT d...
1,2022,AC,6,Deputado Federal,434253,8,54282,REDE / PSOL
2,2022,AC,6,Deputado Federal,434253,8,54282,CIDADANIA / PSDB
3,2018,AC,6,Deputado Federal,417502,8,52188,PT / PSB / PHS / PC do B / DC
4,2018,AC,6,Deputado Federal,417502,8,52188,PDT


## Data Wrangling

### candidatos_df

In [21]:
# drop candidates with null values in the column
candidatos_df.dropna(axis=0, how="any", subset=['NR_IDADE_DATA_POSSE'], inplace=True)
candidatos_df['NR_IDADE_DATA_POSSE'] = candidatos_df['NR_IDADE_DATA_POSSE'].astype(np.int32)

In [22]:
candidatos_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29289 entries, 0 to 29313
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ANO_ELEICAO                 29289 non-null  int64 
 1   DS_ELEICAO                  29289 non-null  object
 2   SG_UE                       29289 non-null  object
 3   DS_CARGO                    29289 non-null  object
 4   SQ_CANDIDATO                29289 non-null  int64 
 5   NR_CANDIDATO                29289 non-null  int64 
 6   NM_CANDIDATO                29289 non-null  object
 7   NR_PARTIDO                  29289 non-null  int64 
 8   SG_PARTIDO                  29289 non-null  object
 9   NM_PARTIDO                  29289 non-null  object
 10  DS_NACIONALIDADE            29289 non-null  object
 11  NR_IDADE_DATA_POSSE         29289 non-null  int32 
 12  DS_GENERO                   29289 non-null  object
 13  DS_GRAU_INSTRUCAO           29289 non-null  ob

In [23]:
candidatos_df.head()

,ANO_ELEICAO,DS_ELEICAO,SG_UE,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DS_NACIONALIDADE,NR_IDADE_DATA_POSSE,DS_GENERO,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_CANDIDATO_INSERIDO_URNA
0,2022,Eleições Gerais Estaduais 2022,DF,DEPUTADO FEDERAL,70001650385,1123,CELY LANGAMER MUNIZ,11,PP,PROGRESSISTAS,BRASILEIRA NATA,51,FEMININO,SUPERIOR COMPLETO,CASADO(A),BRANCA,PROFESSOR DE ENSINO DE PRIMEIRO E SEGUNDO GRAUS,NÃO ELEITO,N,SIM
1,2022,Eleições Gerais Estaduais 2022,PE,DEPUTADO FEDERAL,170001618619,1167,MARCOS BEZERRA CAMPELO,11,PP,PROGRESSISTAS,BRASILEIRA NATA,55,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,OUTROS,SUPLENTE,N,SIM
2,2022,Eleições Gerais Estaduais 2022,RO,DEPUTADO ESTADUAL,220001647615,35355,DIHESSICA DAYANE FERREIRA DA SILVA,35,PMB,PARTIDO DA MULHER BRASILEIRA,BRASILEIRA NATA,25,FEMININO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),PARDA,OUTROS,NÃO ELEITO,N,SIM
3,2022,Eleições Gerais Estaduais 2022,RO,DEPUTADO FEDERAL,220001605554,4456,MARY TERESINHA BRAGANHOL,44,UNIÃO,UNIÃO BRASIL,BRASILEIRA NATA,58,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),BRANCA,SERVIDOR PÚBLICO ESTADUAL,SUPLENTE,N,SIM
4,2022,Eleições Gerais Estaduais 2022,RJ,DEPUTADO ESTADUAL,190001613885,33337,ROBERTA PATRÍCIA DA SILVA ALMEIDA,33,PMN,PARTIDO DA MOBILIZAÇÃO NACIONAL,BRASILEIRA NATA,47,FEMININO,ENSINO FUNDAMENTAL INCOMPLETO,SOLTEIRO(A),BRANCA,DONA DE CASA,SUPLENTE,N,SIM


### receitas_df & despesas_df

In [24]:
# convert column VR_RECEITA from object to float
receitas_df['VR_RECEITA'] = receitas_df['VR_RECEITA'].str.replace(',', '.').astype(float)
despesas_df['VR_DESPESA_CONTRATADA'] = despesas_df['VR_DESPESA_CONTRATADA'].str.replace(',', '.').astype(float)

# convert string to date format
receitas_df['DT_RECEITA'] = pd.to_datetime(receitas_df['DT_RECEITA'], dayfirst = True, format = "%d/%m/%Y")
despesas_df['DT_DESPESA'] = pd.to_datetime(despesas_df['DT_DESPESA'], dayfirst = True, format = "%d/%m/%Y")


In [25]:
receitas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653273 entries, 0 to 653272
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   ANO_ELEICAO              653273 non-null  int64         
 1   DS_ELEICAO               653273 non-null  object        
 2   TP_PRESTACAO_CONTAS      653273 non-null  object        
 3   SQ_PRESTADOR_CONTAS      653273 non-null  int64         
 4   SG_UE                    653273 non-null  object        
 5   NR_CNPJ_PRESTADOR_CONTA  653273 non-null  int64         
 6   DS_CARGO                 653273 non-null  object        
 7   SQ_CANDIDATO             653273 non-null  int64         
 8   NR_CANDIDATO             653273 non-null  int64         
 9   NM_CANDIDATO             653273 non-null  object        
 10  NR_PARTIDO               653273 non-null  int64         
 11  SG_PARTIDO               653273 non-null  object        
 12  NM_PARTIDO      

In [26]:
despesas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1994977 entries, 0 to 1994976
Data columns (total 19 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   ANO_ELEICAO              int64         
 1   DS_ELEICAO               object        
 2   TP_PRESTACAO_CONTAS      object        
 3   SQ_PRESTADOR_CONTAS      int64         
 4   SG_UE                    object        
 5   NR_CNPJ_PRESTADOR_CONTA  int64         
 6   DS_CARGO                 object        
 7   SQ_CANDIDATO             int64         
 8   NR_CANDIDATO             int64         
 9   NM_CANDIDATO             object        
 10  NR_PARTIDO               int64         
 11  SG_PARTIDO               object        
 12  NM_PARTIDO               object        
 13  DS_TIPO_FORNECEDOR       object        
 14  NR_CPF_CNPJ_FORNECEDOR   int64         
 15  NM_FORNECEDOR_RFB        object        
 16  DS_ORIGEM_DESPESA        object        
 17  DT_DESPESA               da

### eleitos_df

In [27]:
header_electeds = [name.upper() for name in eleitos_df.columns]

eleitos_df.rename(dict(zip(list(eleitos_df.columns),header_electeds)), axis=1, inplace=True)


In [28]:
eleitos_df.head()

,DS_CARGO,NR_CANDIDATO,NM_CANDIDATO,SG_PARTIDO,SG_UE,SQ_CANDIDATO,QT_VOTOS_NOM_VALIDOS
0,Deputado Federal,1101,MARIA DO SOCORRO NERI MEDEIROS DE SOUZA,PP,AC,10001621643,25842
1,Deputado Federal,4455,ROZIMEIRE RIBEIRO ANDRADE,UNIÃO,AC,10001612486,21285
2,Deputado Federal,4422,ULYSSES FREITAS PEREIRA DE ARAUJO,UNIÃO,AC,10001612487,21075
3,Deputado Federal,1122,JOSÉ ESTEPHAN BARBARY FILHO,PP,AC,10001621646,19958
4,Deputado Federal,1111,GEHLEN DINIZ ANDRADE,PP,AC,10001621650,19560


### depfed_redes_sociais_df

In [29]:
# retira colunas com dados não relevantes
lista_drop = ['NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'DS_SIT_TOT_TURNO', 'SG_PARTIDO', 'SG_UE', 'QT_VOTOS_NOM_VALIDOS',
              'URL_INSTAGRAM', 'URL_TIKTOK', 'URL_FACEBOOK', 'URL_TWITTER']

# drop colunas
depfed_redsoc_df = depfed_redes_sociais_df.drop(lista_drop, axis=1)

# preenche os vazios com zeros
depfed_redsoc_df = depfed_redsoc_df.fillna(value=0.0, axis=1)

# transforma colunas de float para int
depfed_redsoc_df["SQ_CANDIDATO"] = depfed_redsoc_df["SQ_CANDIDATO"].astype(np.int64)
depfed_redsoc_df["INSTA_FLW"] = depfed_redsoc_df["INSTA_FLW"].astype(np.int64)
depfed_redsoc_df["TK_FLW"] = depfed_redsoc_df["TK_FLW"].astype(np.int64)
depfed_redsoc_df["TK_LIKE"] = depfed_redsoc_df["TK_LIKE"].astype(np.int64)
depfed_redsoc_df["FB_FLW"] = depfed_redsoc_df["FB_FLW"].astype(np.int64)
depfed_redsoc_df["TWT_FLW"] = depfed_redsoc_df["TWT_FLW"].astype(np.int64)

In [30]:
depfed_redsoc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9970 entries, 0 to 9969
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   SQ_CANDIDATO  9970 non-null   int64
 1   INSTA_FLW     9970 non-null   int64
 2   TK_FLW        9970 non-null   int64
 3   TK_LIKE       9970 non-null   int64
 4   FB_FLW        9970 non-null   int64
 5   TWT_FLW       9970 non-null   int64
dtypes: int64(6)
memory usage: 467.5 KB


In [31]:
depfed_redsoc_df.head()

,SQ_CANDIDATO,INSTA_FLW,TK_FLW,TK_LIKE,FB_FLW,TWT_FLW
0,70001615922,1500000,0,0,1500000,1700000
1,170001609080,48400,847,6355,54000,286
2,220001600708,37800,276,511,30000,0
3,190001619522,47000,0,0,35000,3837
4,130001611005,6000000,3500000,48500000,4000000,1000000


### depfed_quociente_eleitoral

In [32]:
# filtra pelo ano de 2022 apenas
depfed_quociente_eleitoral_2022 = depfed_quociente_eleitoral[depfed_quociente_eleitoral["aa_eleicao"] == 2022]

# pega somente colunas de interesse
depfed_quociente_eleitoral_2022 = depfed_quociente_eleitoral_2022.loc[:, ["sg_uf", "qt_vagas_qe", "vr_quociente_eleitoral"]]

#retira duplicatas das linhas
depfed_quociente_eleitoral_2022 = depfed_quociente_eleitoral_2022.drop_duplicates(ignore_index=True)

# inserir o estado de SC que está faltando nos dados
depfed_quociente_eleitoral_2022.loc[len(depfed_quociente_eleitoral_2022.index)] = ["SC", 16, 238252]
depfed_quociente_eleitoral_2022 = depfed_quociente_eleitoral_2022.sort_values("sg_uf", ignore_index=True)
depfed_quociente_eleitoral_2022

,sg_uf,qt_vagas_qe,vr_quociente_eleitoral
0,AC,8,54282
1,AL,9,183418
2,AM,8,247060
3,AP,8,52877
4,BA,39,204062
5,CE,22,232193
6,DF,8,200940
7,ES,10,208443
8,GO,17,202332
9,MA,18,205917


In [33]:
depfed_quociente_eleitoral_2022["qt_vagas_qe"].sum()

513

## Tratamento de dados - candidatos_df

In [34]:
candidatos_df['ST_CANDIDATO_INSERIDO_URNA'].unique().tolist()

['SIM', 'NÃO']

In [35]:
candidatos_df['DS_CARGO'].unique().tolist()

['DEPUTADO FEDERAL',
 'DEPUTADO ESTADUAL',
 '1º SUPLENTE',
 '2º SUPLENTE',
 'GOVERNADOR',
 'SENADOR',
 'DEPUTADO DISTRITAL',
 'VICE-GOVERNADOR',
 'VICE-PRESIDENTE',
 'PRESIDENTE']

In [36]:
# filtro
depfed_cand_df = candidatos_df.loc[(candidatos_df['ST_CANDIDATO_INSERIDO_URNA'] == "SIM")
                              & (candidatos_df['DS_CARGO'] == "DEPUTADO FEDERAL")].reset_index(drop=True, inplace=False)
depfed_cand_df.head()

,ANO_ELEICAO,DS_ELEICAO,SG_UE,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DS_NACIONALIDADE,NR_IDADE_DATA_POSSE,DS_GENERO,DS_GRAU_INSTRUCAO,DS_ESTADO_CIVIL,DS_COR_RACA,DS_OCUPACAO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_CANDIDATO_INSERIDO_URNA
0,2022,Eleições Gerais Estaduais 2022,DF,DEPUTADO FEDERAL,70001650385,1123,CELY LANGAMER MUNIZ,11,PP,PROGRESSISTAS,BRASILEIRA NATA,51,FEMININO,SUPERIOR COMPLETO,CASADO(A),BRANCA,PROFESSOR DE ENSINO DE PRIMEIRO E SEGUNDO GRAUS,NÃO ELEITO,N,SIM
1,2022,Eleições Gerais Estaduais 2022,PE,DEPUTADO FEDERAL,170001618619,1167,MARCOS BEZERRA CAMPELO,11,PP,PROGRESSISTAS,BRASILEIRA NATA,55,MASCULINO,SUPERIOR COMPLETO,CASADO(A),PARDA,OUTROS,SUPLENTE,N,SIM
2,2022,Eleições Gerais Estaduais 2022,RO,DEPUTADO FEDERAL,220001605554,4456,MARY TERESINHA BRAGANHOL,44,UNIÃO,UNIÃO BRASIL,BRASILEIRA NATA,58,FEMININO,SUPERIOR COMPLETO,SOLTEIRO(A),BRANCA,SERVIDOR PÚBLICO ESTADUAL,SUPLENTE,N,SIM
3,2022,Eleições Gerais Estaduais 2022,RJ,DEPUTADO FEDERAL,190001609675,1966,FERNANDO DA SILVA FERNANDES,19,PODE,PODEMOS,BRASILEIRA NATA,68,MASCULINO,ENSINO MÉDIO COMPLETO,CASADO(A),PARDA,OUTROS,SUPLENTE,N,SIM
4,2022,Eleições Gerais Estaduais 2022,MG,DEPUTADO FEDERAL,130001612965,7725,RAFAEL CANDIDO,77,SOLIDARIEDADE,SOLIDARIEDADE,BRASILEIRA NATA,32,MASCULINO,ENSINO MÉDIO COMPLETO,SOLTEIRO(A),BRANCA,EMPRESÁRIO,SUPLENTE,N,SIM


In [37]:
depfed_cand_df['DS_SIT_TOT_TURNO'].unique().tolist()

['NÃO ELEITO', 'SUPLENTE', 'ELEITO POR MÉDIA', 'ELEITO POR QP']

In [38]:
len(depfed_cand_df['SQ_CANDIDATO'].unique().tolist())

9970

In [39]:
depfed_dataset_df = depfed_cand_df.loc[:,
                                       ['SQ_CANDIDATO',
                                        'NM_CANDIDATO',
                                        'DS_CARGO',
                                        'SG_UE',
                                        'NR_PARTIDO',
                                        'SG_PARTIDO',
                                        'NR_IDADE_DATA_POSSE',
                                        'DS_GENERO',
                                        'DS_COR_RACA',
                                        'DS_SIT_TOT_TURNO',
                                        'ST_REELEICAO']
                                      ]

depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO
0,70001650385,CELY LANGAMER MUNIZ,DEPUTADO FEDERAL,DF,11,PP,51,FEMININO,BRANCA,NÃO ELEITO,N
1,170001618619,MARCOS BEZERRA CAMPELO,DEPUTADO FEDERAL,PE,11,PP,55,MASCULINO,PARDA,SUPLENTE,N
2,220001605554,MARY TERESINHA BRAGANHOL,DEPUTADO FEDERAL,RO,44,UNIÃO,58,FEMININO,BRANCA,SUPLENTE,N
3,190001609675,FERNANDO DA SILVA FERNANDES,DEPUTADO FEDERAL,RJ,19,PODE,68,MASCULINO,PARDA,SUPLENTE,N
4,130001612965,RAFAEL CANDIDO,DEPUTADO FEDERAL,MG,77,SOLIDARIEDADE,32,MASCULINO,BRANCA,SUPLENTE,N


### Pocessamento dos dados - candidatos_df

In [40]:
# one hot encoding no atributo ST_REELEICAO
reeleicao_cat_encoder = OneHotEncoder()
reeleicao_1hot = reeleicao_cat_encoder.fit_transform(depfed_dataset_df[["ST_REELEICAO"]]).toarray()
reeleicao_cat_encoder.categories_


[array(['N', 'S'], dtype=object)]

In [41]:
# insere os atributos REELEICAO_NAO/REELEICAO_SIM no dataframe
depfed_dataset_df['REELEICAO'] = reeleicao_1hot[:,1]


In [42]:
# one hot encoding no atributo DS_SIT_TOT_TURNO
situacao_cat_encoder = OneHotEncoder()
situacao_1hot = situacao_cat_encoder.fit_transform(depfed_dataset_df[["DS_SIT_TOT_TURNO"]]).toarray()
situacao_cat_encoder.categories_


[array(['ELEITO POR MÉDIA', 'ELEITO POR QP', 'NÃO ELEITO', 'SUPLENTE'],
       dtype=object)]

In [43]:
# insere os atributos ELEITO/NAO_ELEITO no dataframe
depfed_dataset_df['ST_ELEITORAL'] = np.sum(situacao_1hot, axis=1, keepdims=True, where=[True, True, False, False])


In [44]:
# one hot encoding no atributo DS_COR_RACA
cor_raca_cat_encoder = OneHotEncoder()
cor_raca_1hot = cor_raca_cat_encoder.fit_transform(depfed_dataset_df[["DS_COR_RACA"]]).toarray()
cor_raca_cat_encoder.categories_

[array(['AMARELA', 'BRANCA', 'INDÍGENA', 'NÃO INFORMADO', 'PARDA', 'PRETA'],
       dtype=object)]

In [45]:
# insere os atributos REELEICAO_NAO/REELEICAO_SIM no dataframe
depfed_dataset_df['COR_RACA_AMARELA'] = cor_raca_1hot[:, 0]
depfed_dataset_df['COR_RACA_BRANCA'] = cor_raca_1hot[:, 1]
depfed_dataset_df['COR_RACA_INDIGENA'] = cor_raca_1hot[:, 2]
depfed_dataset_df['COR_RACA_NAO_INFORMADO'] = cor_raca_1hot[:, 3]
depfed_dataset_df['COR_RACA_PARDA'] = cor_raca_1hot[:, 4]
depfed_dataset_df['COR_RACA_PRETA'] = cor_raca_1hot[:, 5]
# depfed_dataset_df.loc[98:102,:]

In [46]:
# one hot encoding no atributo DS_GENERO
genero_cat_encoder = OneHotEncoder()
genero_1hot = genero_cat_encoder.fit_transform(depfed_dataset_df[["DS_GENERO"]]).toarray()
genero_cat_encoder.categories_

[array(['FEMININO', 'MASCULINO'], dtype=object)]

In [47]:
# insere o atributo GENERO no dataframe
depfed_dataset_df['GENERO'] = genero_1hot[:, 1]


### Pocessamento dos dados - receitas_df

In [48]:
receitas_df['DS_CARGO'].unique().tolist()

['Deputado Federal',
 'Deputado Estadual',
 'Deputado Distrital',
 'Senador',
 'Governador',
 'Presidente']

In [49]:
# filtro por Deputado Federal
depfed_receitas_df = receitas_df.loc[receitas_df['DS_CARGO'] == "Deputado Federal"].reset_index(drop=True, inplace=False)


In [50]:
# agrupa as receitas por candidato e fonte da receita
sq_receitas = depfed_receitas_df.groupby(['SQ_CANDIDATO', 'DS_FONTE_RECEITA'])['VR_RECEITA'].sum()

# monta dataset a partir do grupamento
sq_fonte_receitas_df = pd.DataFrame()
sq_fonte_receitas_df['SQ_CANDIDATO'] = sq_receitas.index.get_level_values(0)
sq_fonte_receitas_df['DS_FONTE_RECEITA'] = sq_receitas.index.get_level_values(1)
sq_fonte_receitas_df['VR_RECEITA'] = sq_receitas.values

# tipos de fonte de receita
fonte_lista = sq_fonte_receitas_df['DS_FONTE_RECEITA'].unique().tolist()


In [51]:
# candidato para cada tipo de receita
fonte_listadf = []

for fonte in fonte_lista:
    
    fonte_receita_df = sq_fonte_receitas_df.loc[sq_fonte_receitas_df['DS_FONTE_RECEITA'] == fonte]
    fonte_receita_df = fonte_receita_df.drop(labels=['DS_FONTE_RECEITA'], axis=1)
    fonte_receita_df.columns = list(fonte_receita_df.columns)[:-1] + [fonte]
    fonte_listadf.append(fonte_receita_df)
    

In [52]:
# pega sequencial e nome do dep federal do dataset
sq_depfed = depfed_dataset_df.loc[:, ['SQ_CANDIDATO']].sort_values('SQ_CANDIDATO', ignore_index=True)
sq_depfed.head()

,SQ_CANDIDATO
0,10001595340
1,10001595341
2,10001595342
3,10001595343
4,10001595344


In [53]:
# receita do candidato por fonte
merged_sq_fundoespecial_df = sq_depfed.merge(fonte_listadf[0], how='left', on='SQ_CANDIDATO')
merged_sq_fundoespecial_df = merged_sq_fundoespecial_df.merge(fonte_listadf[1], how='left', on='SQ_CANDIDATO')
merged_sq_fundoespecial_df = merged_sq_fundoespecial_df.merge(fonte_listadf[2], how='left', on='SQ_CANDIDATO')
merged_sq_fundoespecial_df.fillna(0, inplace=True)


In [54]:
# coloca os dados de fonte de receita no dataset
depfed_dataset_df = depfed_dataset_df.merge(merged_sq_fundoespecial_df, how='left', on='SQ_CANDIDATO')
depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO
0,70001650385,CELY LANGAMER MUNIZ,DEPUTADO FEDERAL,DF,11,PP,51,FEMININO,BRANCA,NÃO ELEITO,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,538615.0,0.0,0.0
1,170001618619,MARCOS BEZERRA CAMPELO,DEPUTADO FEDERAL,PE,11,PP,55,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,300000.0
2,220001605554,MARY TERESINHA BRAGANHOL,DEPUTADO FEDERAL,RO,44,UNIÃO,58,FEMININO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,616525.0,243879.0,0.0
3,190001609675,FERNANDO DA SILVA FERNANDES,DEPUTADO FEDERAL,RJ,19,PODE,68,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,110800.0,0.0,0.0
4,130001612965,RAFAEL CANDIDO,DEPUTADO FEDERAL,MG,77,SOLIDARIEDADE,32,MASCULINO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,150000.0,0.0,50000.0


### Pocessamento dos dados - vagas_df

In [55]:
# numero de candidatos a dep federal por UE
cand_depfed_UE = depfed_cand_df.groupby(['SG_UE'])['SG_UE'].count()
cand_depfed_UE

SG_UE
AC     134
AL     161
AM     157
AP     137
BA     693
CE     366
DF     206
ES     192
GO     358
MA     340
MG    1058
MS     151
MT     151
PA     301
PB     227
PE     433
PI     166
PR     610
RJ    1053
RN     176
RO     154
RR     155
RS     524
SC     303
SE     157
SP    1458
TO     149
Name: SG_UE, dtype: int64

In [56]:
# numero de vagas de dep federal por UE
vagas_depfed_UE = vagas_df.loc[vagas_df['DS_CARGO'] =='Deputado Federal']
vagas_depfed_UE = vagas_depfed_UE.groupby(['SG_UE'])['QT_VAGAS'].sum()

# dicionario com as vagas por UE
vagas_depfed_UE_dict = dict(zip(vagas_depfed_UE.index, vagas_depfed_UE.values))

In [57]:
# numero de candidatos a dep federal por vaga, em cada UE
dicio_cand_depfed_por_vaga = dict(zip(vagas_depfed_UE.index.tolist(),
                                      (cand_depfed_UE.values/vagas_depfed_UE.values).tolist()
                                     )
                                 )

In [58]:
# insere o numero de candidatos a dep federal por vaga e em cada UE no dataset
depfed_dataset_df['RAZAO_CAND_VAGA_UE'] = np.nan

for idx in depfed_dataset_df.index:
    
    depfed_dataset_df.at[idx, 'RAZAO_CAND_VAGA_UE'] = dicio_cand_depfed_por_vaga[depfed_dataset_df.at[idx, 'SG_UE']]

depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO,RAZAO_CAND_VAGA_UE
0,70001650385,CELY LANGAMER MUNIZ,DEPUTADO FEDERAL,DF,11,PP,51,FEMININO,BRANCA,NÃO ELEITO,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,538615.0,0.0,0.0,25.750000
1,170001618619,MARCOS BEZERRA CAMPELO,DEPUTADO FEDERAL,PE,11,PP,55,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,300000.0,17.320000
2,220001605554,MARY TERESINHA BRAGANHOL,DEPUTADO FEDERAL,RO,44,UNIÃO,58,FEMININO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,616525.0,243879.0,0.0,19.250000
3,190001609675,FERNANDO DA SILVA FERNANDES,DEPUTADO FEDERAL,RJ,19,PODE,68,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,110800.0,0.0,0.0,22.891304
4,130001612965,RAFAEL CANDIDO,DEPUTADO FEDERAL,MG,77,SOLIDARIEDADE,32,MASCULINO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,150000.0,0.0,50000.0,19.962264


In [59]:
# numero de candidatos a dep federal por partido e por UE
cand_depfed_part_UE = depfed_cand_df.groupby(['SG_PARTIDO','SG_UE'])['SG_UE'].count()

# monta dataset a partir do grupamento
cand_part_UE_df = pd.DataFrame()
cand_part_UE_df['SG_PARTIDO'] = cand_depfed_part_UE.index.get_level_values(0)
cand_part_UE_df['SG_UE'] = cand_depfed_part_UE.index.get_level_values(1)
cand_part_UE_df['NR_CANDIDATOS'] = cand_depfed_part_UE.values

# razao do numero de candidatos a dep federal por partido e por UE
cand_part_UE_df['NR_VAGAS_UE'] = np.nan

for idx in cand_part_UE_df.index:
    
    cand_part_UE_df.at[idx, 'NR_VAGAS_UE'] = vagas_depfed_UE_dict[cand_part_UE_df.at[idx, 'SG_UE']]

cand_part_UE_df['RZ_CAND_VAGAS_UE'] = cand_part_UE_df['NR_CANDIDATOS']/cand_part_UE_df['NR_VAGAS_UE']

# cand_part_UE_df.head()

In [60]:
# insere razao candidatos por partido e por UE no dataset
depfed_dataset_df['RAZAO_CANDPART_VAGAS_UE'] = np.nan

for idx in depfed_dataset_df.index:
    
    ue = depfed_dataset_df.at[idx, 'SG_UE']
    partido = depfed_dataset_df.at[idx, 'SG_PARTIDO']
    
    df_filt = cand_part_UE_df.loc[cand_part_UE_df['SG_PARTIDO'] == partido]
    dicio_filt = dict(zip(df_filt['SG_UE'],df_filt['RZ_CAND_VAGAS_UE']))
    
    depfed_dataset_df.at[idx, 'RAZAO_CANDPART_VAGAS_UE'] = dicio_filt[ue]

depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO,RAZAO_CAND_VAGA_UE,RAZAO_CANDPART_VAGAS_UE
0,70001650385,CELY LANGAMER MUNIZ,DEPUTADO FEDERAL,DF,11,PP,51,FEMININO,BRANCA,NÃO ELEITO,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,538615.0,0.0,0.0,25.750000,1.125000
1,170001618619,MARCOS BEZERRA CAMPELO,DEPUTADO FEDERAL,PE,11,PP,55,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,300000.0,17.320000,0.840000
2,220001605554,MARY TERESINHA BRAGANHOL,DEPUTADO FEDERAL,RO,44,UNIÃO,58,FEMININO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,616525.0,243879.0,0.0,19.250000,1.125000
3,190001609675,FERNANDO DA SILVA FERNANDES,DEPUTADO FEDERAL,RJ,19,PODE,68,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,110800.0,0.0,0.0,22.891304,1.021739
4,130001612965,RAFAEL CANDIDO,DEPUTADO FEDERAL,MG,77,SOLIDARIEDADE,32,MASCULINO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,150000.0,0.0,50000.0,19.962264,0.679245


### Pocessamento dos dados - relação com candidatos presidência

In [61]:
# 5 mais bem posicionados nas pesquisas eleitorais presidenciais por partido
part_pres_pos_dicio = {"PT": .48, "PL": .40, "PDT": .08, "MDB": .05, "UNIÃO": .02}

In [62]:
# insere os valores no dataset
depfed_dataset_df['PART_PRES_POS'] = np.nan

for idx in depfed_dataset_df.index:
    
    sg_partido = depfed_dataset_df.at[idx, 'SG_PARTIDO']
    
    if sg_partido in part_pres_pos_dicio.keys():
        
        depfed_dataset_df.at[idx, 'PART_PRES_POS'] = part_pres_pos_dicio[sg_partido]
       
    else:
        
        depfed_dataset_df.at[idx, 'PART_PRES_POS'] = 0.0

depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO,RAZAO_CAND_VAGA_UE,RAZAO_CANDPART_VAGAS_UE,PART_PRES_POS
0,70001650385,CELY LANGAMER MUNIZ,DEPUTADO FEDERAL,DF,11,PP,51,FEMININO,BRANCA,NÃO ELEITO,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,538615.0,0.0,0.0,25.750000,1.125000,0.00
1,170001618619,MARCOS BEZERRA CAMPELO,DEPUTADO FEDERAL,PE,11,PP,55,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,300000.0,17.320000,0.840000,0.00
2,220001605554,MARY TERESINHA BRAGANHOL,DEPUTADO FEDERAL,RO,44,UNIÃO,58,FEMININO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,616525.0,243879.0,0.0,19.250000,1.125000,0.02
3,190001609675,FERNANDO DA SILVA FERNANDES,DEPUTADO FEDERAL,RJ,19,PODE,68,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,110800.0,0.0,0.0,22.891304,1.021739,0.00
4,130001612965,RAFAEL CANDIDO,DEPUTADO FEDERAL,MG,77,SOLIDARIEDADE,32,MASCULINO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,150000.0,0.0,50000.0,19.962264,0.679245,0.00


### Pocessamento dos dados - depfed_redes_sociais_df

In [63]:
# coloca os dados de redes sociais no dataset
depfed_dataset_df = depfed_dataset_df.merge(depfed_redsoc_df, how='left', on="SQ_CANDIDATO")
depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO,RAZAO_CAND_VAGA_UE,RAZAO_CANDPART_VAGAS_UE,PART_PRES_POS,INSTA_FLW,TK_FLW,TK_LIKE,FB_FLW,TWT_FLW
0,70001650385,CELY LANGAMER MUNIZ,DEPUTADO FEDERAL,DF,11,PP,51,FEMININO,BRANCA,NÃO ELEITO,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,538615.0,0.0,0.0,25.750000,1.125000,0.00,716,0,0,716,0
1,170001618619,MARCOS BEZERRA CAMPELO,DEPUTADO FEDERAL,PE,11,PP,55,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,300000.0,17.320000,0.840000,0.00,671,0,0,671,0
2,220001605554,MARY TERESINHA BRAGANHOL,DEPUTADO FEDERAL,RO,44,UNIÃO,58,FEMININO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,616525.0,243879.0,0.0,19.250000,1.125000,0.02,2635,0,0,2635,0
3,190001609675,FERNANDO DA SILVA FERNANDES,DEPUTADO FEDERAL,RJ,19,PODE,68,MASCULINO,PARDA,SUPLENTE,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,110800.0,0.0,0.0,22.891304,1.021739,0.00,1575,0,0,1575,0
4,130001612965,RAFAEL CANDIDO,DEPUTADO FEDERAL,MG,77,SOLIDARIEDADE,32,MASCULINO,BRANCA,SUPLENTE,N,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,150000.0,0.0,50000.0,19.962264,0.679245,0.00,198,0,0,198,0


In [64]:
depfed_dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9970 entries, 0 to 9969
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   SQ_CANDIDATO             9970 non-null   int64  
 1   NM_CANDIDATO             9970 non-null   object 
 2   DS_CARGO                 9970 non-null   object 
 3   SG_UE                    9970 non-null   object 
 4   NR_PARTIDO               9970 non-null   int64  
 5   SG_PARTIDO               9970 non-null   object 
 6   NR_IDADE_DATA_POSSE      9970 non-null   int32  
 7   DS_GENERO                9970 non-null   object 
 8   DS_COR_RACA              9970 non-null   object 
 9   DS_SIT_TOT_TURNO         9970 non-null   object 
 10  ST_REELEICAO             9970 non-null   object 
 11  REELEICAO                9970 non-null   float64
 12  ST_ELEITORAL             9970 non-null   float64
 13  COR_RACA_AMARELA         9970 non-null   float64
 14  COR_RACA_BRANCA         

### Pocessamento dos dados - depfed_quociente_eleitoral

In [65]:
# renomeia as colunas
depfed_quociente_eleitoral_2022.rename(columns={"sg_uf": "SG_UE",
                                                "qt_vagas_qe": "QT_VAGAS_UE",
                                                "vr_quociente_eleitoral": "QUOCIENTE_ELEITORAL_UE"},
                                      inplace=True)

# dicionarios com ue como chaves e vagas e QE como valores
dic_ue_vagas = dict(zip(depfed_quociente_eleitoral_2022["SG_UE"],depfed_quociente_eleitoral_2022["QT_VAGAS_UE"]))
dic_ue_qe = dict(zip(depfed_quociente_eleitoral_2022["SG_UE"],depfed_quociente_eleitoral_2022["QUOCIENTE_ELEITORAL_UE"]))

depfed_quociente_eleitoral_2022.head()

,SG_UE,QT_VAGAS_UE,QUOCIENTE_ELEITORAL_UE
0,AC,8,54282
1,AL,9,183418
2,AM,8,247060
3,AP,8,52877
4,BA,39,204062


In [66]:
# coloca os dados de vagas e quociente eleitoral por UE
depfed_dataset_df.loc[:, ["QT_VAGAS_UE","QUOCIENTE_ELEITORAL_UE"]] = np.nan

for idx in depfed_dataset_df.index:
    
    UE = depfed_dataset_df.at[idx, "SG_UE"]
    depfed_dataset_df.at[idx, "QT_VAGAS_UE"] = dic_ue_vagas[UE]
    depfed_dataset_df.at[idx, "QUOCIENTE_ELEITORAL_UE"] = dic_ue_qe[UE]


In [67]:
depfed_dataset_df.tail()

,SQ_CANDIDATO,NM_CANDIDATO,DS_CARGO,SG_UE,NR_PARTIDO,SG_PARTIDO,NR_IDADE_DATA_POSSE,DS_GENERO,DS_COR_RACA,DS_SIT_TOT_TURNO,ST_REELEICAO,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO,RAZAO_CAND_VAGA_UE,RAZAO_CANDPART_VAGAS_UE,PART_PRES_POS,INSTA_FLW,TK_FLW,TK_LIKE,FB_FLW,TWT_FLW,QT_VAGAS_UE,QUOCIENTE_ELEITORAL_UE
9965,190001606459,CARLA CRISTINA MOTTA PEIXOTO,DEPUTADO FEDERAL,RJ,33,PMN,42,FEMININO,PRETA,NÃO ELEITO,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,50000.00,0.0,0.0,22.891304,1.021739,0.0,517,0,0,517,0,46.0,186435.0
9966,130001645109,VALDEIR GERALDO BATISTA,DEPUTADO FEDERAL,MG,36,AGIR,55,MASCULINO,PARDA,NÃO ELEITO,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.00,0.0,0.0,19.962264,0.358491,0.0,945,0,0,945,0,53.0,210964.0
9967,140001615724,ARNALDO JORDY FIGUEIREDO,DEPUTADO FEDERAL,PA,23,CIDADANIA,64,MASCULINO,PARDA,NÃO ELEITO,N,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,2838884.00,16700.0,100000.0,17.705882,0.176471,0.0,9434,3098,296,28000,8681,17.0,265972.0
9968,120001650736,JORGE MARTINHO,DEPUTADO FEDERAL,MS,55,PSD,63,MASCULINO,PRETA,NÃO ELEITO,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,310000.00,29000.0,0.0,18.875000,1.125000,0.0,5179,0,0,5179,0,8.0,169608.0
9969,160001621994,MESAEL CAETANO DOS SANTOS,DEPUTADO FEDERAL,PR,18,REDE,57,MASCULINO,PRETA,NÃO ELEITO,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,72252.96,500.0,0.0,20.333333,0.700000,0.0,78,0,0,78,0,30.0,201288.0


### Dropa colunas não relevantes no dataset

In [68]:
# retira as colunas já tratadas
depfed_dataset_df.drop(['DS_CARGO',
                        'SG_UE',
                        'NR_PARTIDO',
                        'SG_PARTIDO',
                        'DS_GENERO',
                        'DS_COR_RACA',
                        'DS_SIT_TOT_TURNO',
                        'ST_REELEICAO'],
                       axis=1,
                       inplace=True
                      )

In [69]:
depfed_dataset_df.head()

,SQ_CANDIDATO,NM_CANDIDATO,NR_IDADE_DATA_POSSE,REELEICAO,ST_ELEITORAL,COR_RACA_AMARELA,COR_RACA_BRANCA,COR_RACA_INDIGENA,COR_RACA_NAO_INFORMADO,COR_RACA_PARDA,COR_RACA_PRETA,GENERO,FUNDO ESPECIAL,OUTROS RECURSOS,FUNDO PARTIDARIO,RAZAO_CAND_VAGA_UE,RAZAO_CANDPART_VAGAS_UE,PART_PRES_POS,INSTA_FLW,TK_FLW,TK_LIKE,FB_FLW,TWT_FLW,QT_VAGAS_UE,QUOCIENTE_ELEITORAL_UE
0,70001650385,CELY LANGAMER MUNIZ,51,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,538615.0,0.0,0.0,25.750000,1.125000,0.00,716,0,0,716,0,8.0,200940.0
1,170001618619,MARCOS BEZERRA CAMPELO,55,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,300000.0,17.320000,0.840000,0.00,671,0,0,671,0,25.0,198795.0
2,220001605554,MARY TERESINHA BRAGANHOL,58,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,616525.0,243879.0,0.0,19.250000,1.125000,0.02,2635,0,0,2635,0,8.0,108643.0
3,190001609675,FERNANDO DA SILVA FERNANDES,68,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,110800.0,0.0,0.0,22.891304,1.021739,0.00,1575,0,0,1575,0,46.0,186435.0
4,130001612965,RAFAEL CANDIDO,32,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,150000.0,0.0,50000.0,19.962264,0.679245,0.00,198,0,0,198,0,53.0,210964.0


In [70]:
depfed_dataset_df.shape

(9970, 25)

### Download dataset

In [71]:
depfed_dataset_df.to_csv(r'./dataset/depfed_dataset.csv',
                         encoding = 'utf-8-sig',
                         index = False)